In [1]:
#import packages
import pandas as pd
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
#modify import path
import sys
sys.path.append('../our_analysis')

#import our code
from hybrid_mle import fit_hybrid_mixed_model, fit_hybrid_mixed_dynamic_model


In [3]:
#Set WD
notebook_dir = os.getcwd()

In [ ]:
#load dataset
# df = pd.read_csv("../dataset/all_subjects.csv") #our dataset
df = pd.read_csv("../dataset/beh_noslow.csv") #i'm fitting to replicate the paper.


## some notes about using all_subjects:
# If you get errors you may need to rename the fields of allsubjects becuase the original paper dataset (beh_noslow.csv) and our dataset (all_subjects) have slightly different capitalizations of headers.
# If you get additional errors filter out the NaNs from the df before sending them to the processing function. 

#MLE takes time the first simple run was quite fast (~10min), but expect longer depending on your CPU and the complexity of the model you're running

story_trials = df[df["condition"] == "story"]
abstract_trials=df[df["condition"] == "abstract"]

# Fit Hybrid Model
We're currently fitting using the provided stan model. I made modifications to ensure we could just pass dfs instead of raw csvs. 

We are using a mixed effects approach to estimate the weight allowing it to vary across participants. Not necessary as we could fix all effects (the pset is unclear what they're really asking for here), but this provides more nuance to the analysis. 

Note we likely won't be very close to the reported table in the paper as estimated individually and then pooled across all participants.

We can rerun this to get results closer to the paper by setting the priors in the stan model. Look at the upper/lower parameter bounds in the stan file (sdn_hw2/our_analysis/hybrid_mixed.stan). We want a decent width for the parameter search but we can make them a bit closer to the actual paper. @TODO by anyone: it would be good to check the supplemental to see if they added a note about what the priors were

In [ ]:
story_results_df, params = fit_hybrid_mixed_model(
    data_df=story_trials,
    stan_file="../our_analysis/hybrid_mixed.stan",
    output_file="story_params.csv"
)

abstract_results_df, params = fit_hybrid_mixed_model(
    data_df=abstract_trials,
    stan_file="../our_analysis/hybrid_mixed.stan",
    output_file="abstract_params.csv"
)

In [24]:
#store the results in a clean table we can use for the assignment
results =[] 

for df in [story_results_df,abstract_results_df]:
    params=df.drop(columns='w').iloc[0].to_dict()
    params['w_mean'] = df['w'].mean()
    params['w_std'] = df['w'].std()
    results.append(params)

results_df = pd.DataFrame(results)
results_df.head()
## we should probably just save this as a csv here. It has the participant field which we can drop in later formatting.
results_df.to_csv("hybrid_model_params.csv", index=False)

# Let's do some ablation:

Here we're going to start fitting with fixed parameters. To do this I added a modified fit function that allows us to prefix certain parameters. This should work to finish the next few parts

In [ ]:
# PARAM_NAMES = ('alpha1', 'alpha2', 'lmbd', 'beta1', 'beta2', 'p', 'w')


## USE THE PARAM NAMES ABOVE. ONLY INPUT THE PARAMS you are FIXING and the value you are fixing it to. 
fixed_param = {
    'lmbd':1
}

story_results_df, params, logli = fit_hybrid_mixed_dynamic_model(
    data_df=story_trials, #note the trial type
    stan_file="../our_analysis/dynamic_hybrid_mixed.stan",
    output_file="story_params.csv",   ##change the output csv name so you don't overwrite your work
    fixed_params = fixed_param,
    return_logli = True
)
